In [1]:
import pandas as pd
pd.set_option('display.max_rows', None)  # Set to None to display all rows
import urllib
import psycopg2
from psycopg2.extras import execute_values
from datetime import datetime, timedelta, timezone

from utils import get_request
from db_utils.base_utils import get_db_connection
from db_utils.prop_odds_db_utils import *

In [2]:
# Today's date (EST)
(datetime.now() - timedelta(hours=6)).strftime('%Y-%m-%d')

'2024-12-13'

In [3]:
# game_id = "c1fa7d3f30fdb408b78917509d1633c3"
# data = fetch_game_markets(game_id, 'player_shots_over_under' )
# data
# for sportsbook in data['sportsbooks']:
#     print(sportsbook)
# # Create empty dict to store outcomes by bookie
# outcomes_by_bookie = {}

# for sportsbook in data['sportsbooks']:
#     bookie_key = sportsbook['bookie_key']
#     outcomes_by_bookie[bookie_key] = []
    
#     # Get all outcomes for this sportsbook
#     for outcome in sportsbook['market']['outcomes']:
#         outcomes_by_bookie[bookie_key].append({
#             'timestamp': outcome['timestamp'],
#             'handicap': outcome['handicap'], 
#             'odds': outcome['odds'],
#             'participant_name': outcome['participant_name'],
#             'name': outcome['name'],
#             'description': outcome['description']
#         })

In [4]:
# fetch_and_store_nhl_games('2024-12-11')

In [5]:
get_nhl_games_from_db('2024-12-11', enable_logging=True)

INFO:root:Retrieved 2 games from the database for date 2024-12-11.


[{'id': 401688056,
  'game_id': '04a7cd0b6c4e5c8cda4d7b79c066c0f4',
  'away_team': 'New York Rangers',
  'home_team': 'Buffalo Sabres',
  'start_timestamp': datetime.datetime(2024, 12, 11, 18, 0, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=64800)))},
 {'id': 401688057,
  'game_id': '636f2219c1642cab1ade8fea22d865ec',
  'away_team': 'Anaheim Ducks',
  'home_team': 'Ottawa Senators',
  'start_timestamp': datetime.datetime(2024, 12, 11, 18, 30, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=64800)))}]

In [6]:
df = pd.read_csv('data/player_shots_ou.csv')
df['game_id'].unique()

array(['c1fa7d3f30fdb408b78917509d1633c3',
       'ca040c2d8fbe567334ed450ab0a02173',
       'd5a68b58d97f6174dc4c540287ff7e3d',
       'af1551ae76d93d6f65c47cb6b26cfd83',
       'c91477501d6ee060317f9fe60db4117f',
       'ba8a3041639b65d6d085fb72d73b7b98',
       '8fc6f715c673c865fed0d1139b31e261',
       '7fcf37fe5624af32decff26b0cf6e1b1',
       'cebb8cb7b5ffee324b463b8c647ce976',
       'ea5343d38a02fc19a5f435c651d245c9',
       '24083d2c612b7f3c8909dcc6bfd9a4e1',
       '480fa6c594877fff4ce30641dc424fe7',
       '77f4c3b4372e22363655c9862424f6b0',
       'c906c0d63116e928c7813733c806c69a',
       'ba3bf40aeee9beb78786af904b0b7e52'], dtype=object)

In [8]:
process_game_markets('2024-12-12', 'TOR', 'player_shots_over_under')

In [12]:
def process_nhl_games_for_date(date, market='player_shots_over_under'):
    games = get_nhl_games_from_db(date)
    if not games:
        fetch_and_store_nhl_games(date)
        games = get_nhl_games_from_db(date)
    game_ids = [game['game_id'] for game in games]
    for game_id in game_ids:
        market_data = fetch_game_markets(game_id, market)
        process_game_markets(game_id, market_data)

In [13]:
process_nhl_games_for_date('2024-12-10')

Inserted/Updated 11 records into prop_odds.game_info table.
Unsupported bookmaker: betopenly
Unsupported bookmaker: betopenly
Unsupported bookmaker: betopenly


In [23]:
def get_player_shots_ou_odds(player_name=None, team_abbreviation=None, query_date=None):
    """
    Retrieve player shot over/under odds from the PostgreSQL prop_odds database
    based on a specific player name or team abbreviation for a given date.
    Args:
        player_name (str, optional): The full name of the player to filter odds by.
        team_abbreviation (str, optional): The abbreviation of the team to filter odds by.
        query_date (str, optional): The date to query in 'YYYY-MM-DD' format. Defaults to today.
    Returns:
        list: A list of dictionaries containing player shot OU odds.
    """

    # Validate that at least one filter is provided
    if not player_name and not team_abbreviation:
        print("Either player_name or team_abbreviation must be provided.")
        return []

    # Determine the query date
    if query_date is None:
        query_date = get_today_date()
    else:
        try:
            query_date = datetime.strptime(query_date, '%Y-%m-%d')
        except ValueError:
            print("Invalid date format. Please use 'YYYY-MM-DD'.")
            return []

    try:
        # Establish a database connection
        conn, cursor = get_db_connection('PROP_ODDS_DB_')
        if not conn or not cursor:
            print("Database connection failed.")
            return []

        # Initialize the base query
        base_query = """
            SELECT 
                pso.game_id,
                pso.sportsbook,
                pso.player,
                pso.ou,
                pso.handicap,
                pso.odds,
                pso.timestamp
            FROM player_shots_ou pso
            JOIN game_info gi ON pso.game_id = gi.game_id
            WHERE gi.start_timestamp::date = %s
        """

        # Initialize parameters list with the query date
        params = [query_date.strftime('%Y-%m-%d')]

        # Add filters based on provided arguments
        if player_name and team_abbreviation:
            # If both player and team filters are provided
            base_query += """
                AND pso.player ILIKE %s
                AND (gi.away_team ILIKE %s OR gi.home_team ILIKE %s)
            """
            params.extend([player_name, team_abbreviation, team_abbreviation])
        elif player_name:
            # If only player filter is provided
            base_query += " AND pso.player ILIKE %s"
            params.append(player_name)
        else:
            # If only team filter is provided
            base_query += " AND (gi.away_team ILIKE %s OR gi.home_team ILIKE %s)"
            params.extend([team_abbreviation, team_abbreviation])

        # Debugging: Print the final SQL query with parameters
        print("Executing SQL Query:")
        print(cursor.mogrify(base_query, tuple(params)).decode('utf-8'))

        # Execute the query with parameters
        cursor.execute(base_query, tuple(params))
        rows = cursor.fetchall()

        # Debugging: Print the number of rows fetched
        print(f"Number of rows fetched: {len(rows)}")

        # Process the retrieved rows
        odds_list = []
        for row in rows:
            odds = {
                'game_id': row[0],
                'sportsbook': row[1],
                'player': row[2],
                'ou': row[3],
                'handicap': row[4],
                'odds': row[5],
                'timestamp': row[6]
            }
            odds_list.append(odds)

        return odds_list

    except psycopg2.Error as e:
        print("A database error occurred:", e)
        return []
    except Exception as e:
        print("An unexpected error occurred:", e)
        return []
    finally:
        # Ensure the cursor and connection are closed
        if cursor:
            cursor.close()
        if conn:
            conn.close()

In [27]:
odds = get_player_shots_ou_odds(player_name='Artturi Lehkonen', query_date='2024-12-10')
odds

Executing SQL Query:

            SELECT 
                pso.game_id,
                pso.sportsbook,
                pso.player,
                pso.ou,
                pso.handicap,
                pso.odds,
                pso.timestamp
            FROM player_shots_ou pso
            JOIN game_info gi ON pso.game_id = gi.game_id
            WHERE gi.start_timestamp::date = '2024-12-10'
         AND pso.player ILIKE 'Artturi Lehkonen'
Number of rows fetched: 39


[{'game_id': 'ba8a3041639b65d6d085fb72d73b7b98',
  'sportsbook': 'barstool',
  'player': 'Artturi Lehkonen',
  'ou': 'Over',
  'handicap': 1.5,
  'odds': -320,
  'timestamp': datetime.datetime(2024, 12, 11, 0, 8, 42, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=64800)))},
 {'game_id': 'ba8a3041639b65d6d085fb72d73b7b98',
  'sportsbook': 'barstool',
  'player': 'Artturi Lehkonen',
  'ou': 'Over',
  'handicap': 2.5,
  'odds': 100,
  'timestamp': datetime.datetime(2024, 12, 11, 0, 8, 42, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=64800)))},
 {'game_id': 'ba8a3041639b65d6d085fb72d73b7b98',
  'sportsbook': 'barstool',
  'player': 'Artturi Lehkonen',
  'ou': 'Over',
  'handicap': 3.5,
  'odds': 250,
  'timestamp': datetime.datetime(2024, 12, 11, 0, 8, 42, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=64800)))},
 {'game_id': 'ba8a3041639b65d6d085fb72d73b7b98',
  'sportsbook': 'barstool',
  'player': 'Artturi Lehkonen',
  'ou': 'Under',
  'handic